<a href="https://colab.research.google.com/github/Ok-subin/OCR-2020_Capstone-Design-/blob/master/EMNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/My\ Drive/Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'1.2.2'

In [ ]:
!pip uninstall keras
!pip install keras==1.2.2

In [2]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.15.0

In [5]:
import gzip
import numpy
import pickle
import os
from keras.models import save_model
import argparse
import numpy
import tensorflow.keras as keras

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.applications.resnet50 import ResNet50
from keras import optimizers


def _read32(bytestream):
    dt = numpy.dtype(numpy.uint32).newbyteorder('>')
    return numpy.frombuffer(bytestream.read(4), dtype=dt)[0]


def extract_images(filename):
    with gzip.open(filename) as bytestream:
        magic = _read32(bytestream)
        if magic != 2051:
            raise ValueError(
                'Invalid magic number %d in MNIST image file: %s' %
                (magic, filename))
        num_images = _read32(bytestream)
        rows = _read32(bytestream)
        cols = _read32(bytestream)
        buf = bytestream.read(rows * cols * num_images)
        data = numpy.frombuffer(buf, dtype=numpy.uint8)
        data = data.reshape(num_images, rows, cols, 1)
        return data


def extract_labels(filename):
    with gzip.open(filename) as bytestream:
        magic = _read32(bytestream)
        if magic != 2049:
            raise ValueError(
              'Invalid magic number %d in MNIST label file: %s' %
              (magic, filename))
        num_items = _read32(bytestream)
        buf = bytestream.read(num_items)
        labels = numpy.frombuffer(buf, dtype=numpy.uint8)
        return labels


def read_emnist(emnist_dir):
    TRAIN_IMAGES = emnist_dir+'/emnist-bymerge-train-images-idx3-ubyte.gz'
    TRAIN_LABELS = emnist_dir+'/emnist-bymerge-train-labels-idx1-ubyte.gz'
    TEST_IMAGES = emnist_dir+'/emnist-bymerge-test-images-idx3-ubyte.gz'
    TEST_LABELS = emnist_dir+'/emnist-bymerge-test-labels-idx1-ubyte.gz'
    MAPPING = emnist_dir+'/emnist-bymerge-mapping.txt'

    train_images = extract_images(TRAIN_IMAGES)
    train_labels = extract_labels(TRAIN_LABELS)
    test_images = extract_images(TEST_IMAGES)
    test_labels = extract_labels(TEST_LABELS)

    with open(MAPPING, "r") as f:
        mapping = f.readlines()
        mapping = {str(x.split()[0]): str(x.split()[1]) for x in mapping}

    train_images = train_images.astype('float32')
    test_images = test_images.astype('float32')

    train_images /= 255
    test_images /= 255

    # Output : (28, 28, 1)
    return ((train_images, train_labels), (test_images, test_labels), mapping)


def save(model, mapping, model_name):
    os.makedirs(os.path.dirname('./modelSave/'), exist_ok=True)
    model_yaml_path = './modelSave/model.yaml'
    mapping_model_path = './modelSave/model_mapping.p'
    model_h5_path = './modelSave/model.h5'

    model_yaml = model.to_yaml()
    with open(model_yaml_path, "w") as yaml_file:
        yaml_file.write(model_yaml)

    save_model(model, model_h5_path)

    model_json = model.to_json()
    with open('./modelSave/model.json', 'w') as json_file:
      json_file.write(model_json)
    model.save_weights("./modelSave/model_weight.h5")

    pickle.dump(mapping, open(mapping_model_path, 'wb'))

    return


K.set_image_dim_ordering('th')


def build_net(training_data, model_name='model', epochs=200):
    (x_train, y_train), (x_test, y_test), mapping = training_data

    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)

    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    num_classes = y_test.shape[1]

    # -----------------------------ㅡ model -----------------------------------------

    model = Sequential()

    model.add(Convolution2D(32, 3, 3, input_shape = (1, 28, 28), activation='relu')) 
    model.add(Convolution2D(32, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(47, activation='softmax')) 

    # -----------------------------ㅡ model end------------------------------------

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(0.001), metrics=['accuracy'])

    model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=100, verbose=1)
    print(model.summary())

    scores = model.evaluate(x_test, y_test, batch_size = 500, verbose=0)
    print("Baseline Error: %.2f%%" % (100-scores[1]*100))

    save(model, mapping, model_name)
    return


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-f', '--file', type=str, help='Path to .mat file')
    parser.add_argument('--epochs', type=int, default=1, help='Number of epochs to train on')
    parser.add_argument('-m', '--model', type=str, help='Model name')
    #args = parser.parse_args() --> error 발생
    args, unknown = parser.parse_known_args()

    training_data = read_emnist("./gzip")
    model = build_net(training_data, args.model, epochs=args.epochs)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 697932 samples, validate on 116323 samples
Epoch 1/1





697932/697932 [==============================] - 1625s - loss: 0.6091 - acc: 0.8064 - val_loss: 0.3295 - val_acc: 0.8826
